In [1]:
import logging
import os
import sys
import tempfile
from glob import glob

import nibabel as nib
import numpy as np
import torch
import glob
from ignite.metrics import Accuracy

import monai
from monai.apps import get_logger
from monai.data import create_test_image_3d
from monai.networks.layers import Norm
from monai.engines import SupervisedEvaluator, SupervisedTrainer
from monai.handlers import (
    CheckpointSaver,
    EarlyStopHandler,
    LrScheduleHandler,
    MeanDice,
    StatsHandler,
    TensorBoardImageHandler,
    TensorBoardStatsHandler,
    ValidationHandler,
    from_engine,
)
from monai.inferers import SimpleInferer, SlidingWindowInferer
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.networks.nets import UNet
from monai.transforms import (
    Activationsd,
    AsChannelFirstd,
    Orientationd,
    EnsureChannelFirstd,
    AsDiscreted,
    Spacingd,
    Compose,
    KeepLargestConnectedComponentd,
    LoadImaged,
    ScaleIntensityRanged,
    CropForegroundd,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
)

train_dir  = 'C:/Users/Hripsime/OneDrive - ABYS MEDICAL/projects/CTPelvic1K_data/train_dir'

def main(train_dir):
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    get_logger("train_log")

    train_images = sorted(glob.glob(os.path.join(train_dir, "*data.nii.gz")))
    train_labels = sorted(glob.glob(os.path.join(train_dir, "*mask_4label.nii.gz")))
    data_dicts = [{"image": image_name, "mask": label_name} for image_name, label_name in zip(train_images, train_labels)]
    train_files, val_files = data_dicts[:-10], data_dicts[-10:]
    
    
    # define transforms for image and segmentation
    train_transforms = Compose(
        [
            LoadImaged(keys=["image", "mask"]),
            EnsureChannelFirstd(keys=["image", "mask"]),
            Orientationd(keys=["image", "mask"], axcodes="RAS"),
            Spacingd(keys=["image", "mask"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
            ScaleIntensityRanged(keys=["image"], a_min=-120, a_max=360,b_min=0.0, b_max=1.0, clip=True,),
            CropForegroundd(keys=["image", "mask"], source_key="image"),
            RandCropByPosNegLabeld(
                keys=["image", "mask"],
                label_key="mask",
                spatial_size=(128, 128, 128),
                pos=1,
                neg=1,
                num_samples=4,
                image_key="image",
                image_threshold=0,
        ),
            EnsureTyped(keys=["image", "mask"]),
        ]
    )
    
    val_transforms = Compose(
        [
            LoadImaged(keys=["image", "mask"]),
            EnsureChannelFirstd(keys=["image", "mask"]),
            Orientationd(keys=["image", "mask"], axcodes="RAS"),
            Spacingd(keys=["image", "mask"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
            ScaleIntensityRanged(keys=["image"], a_min=-120, a_max=360,b_min=0.0, b_max=1.0, clip=True,),
            CropForegroundd(keys=["image", "mask"], source_key="image"),
            EnsureTyped(keys=["image", "mask"]),
        ]
    )

    # create a training data loader
    train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=0)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=0)

    # create a validation data loader
    val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)
    
    
    # create UNet, DiceLoss and Adam optimizer
    device=torch.device("cuda:0")  
    net = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=5,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
        norm=Norm.BATCH,
    ).to(device)
    loss = monai.losses.DiceCELoss(to_onehot_y=True, softmax=True)
    opt = torch.optim.Adam(net.parameters(), 2e-4)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.1)

    val_post_transforms = Compose(
        [
            EnsureTyped(keys="pred"),
            Activationsd(keys="pred", sigmoid=True),
            AsDiscreted(keys="pred", threshold=0.5),
            KeepLargestConnectedComponentd(keys="pred", applied_labels=[5]),
        ]
    )
    val_handlers = [
        # apply “EarlyStop” logic based on the validation metrics
        EarlyStopHandler(trainer=None, patience=20, score_function=lambda x: x.state.metrics["val_mean_dice"]),
        # use the logger "train_log" defined at the beginning of this program
        StatsHandler(name="train_log", output_transform=lambda x: None),
        TensorBoardStatsHandler(log_dir="./runs/", output_transform=lambda x: None),
        TensorBoardImageHandler(
            log_dir="./runs/",
            batch_transform=from_engine(["image", "mask"]),
            output_transform=from_engine(["pred"]),
        ),
        CheckpointSaver(save_dir="./runs/", save_dict={"net": net}, save_key_metric=True),
    ]

    evaluator = SupervisedEvaluator(
        device=device,
        val_data_loader=val_loader,
        network=net,
        inferer=SlidingWindowInferer(roi_size=(160, 160, 160), sw_batch_size=4, overlap=0.5),
        postprocessing=val_post_transforms,
        key_val_metric={
            "val_mean_dice": MeanDice(include_background=True, output_transform=from_engine(["pred", "mask"]))
        },
        additional_metrics={"val_acc": Accuracy(output_transform=from_engine(["pred", "mask"]))},
        val_handlers=val_handlers,
        # if no FP16 support in GPU or PyTorch version < 1.6, will not enable AMP evaluation
        amp=True if monai.utils.get_torch_version_tuple() >= (1, 6) else False,
    )

    train_post_transforms = Compose(
        [
            Activationsd(keys="pred", sigmoid=True),
            AsDiscreted(keys="pred", threshold=0.5),
            KeepLargestConnectedComponentd(keys="pred", applied_labels=[5]),
        ]
    )

    train_handlers = [
        # apply “EarlyStop” logic based on the loss value, use “-” negative value because smaller loss is better
        EarlyStopHandler(trainer=None, patience=20, score_function=lambda x: -x.state.output[0]["loss"], epoch_level=False),
        LrScheduleHandler(lr_scheduler=lr_scheduler, print_lr=True),
        ValidationHandler(validator=evaluator, interval=1, epoch_level=True),
        # use the logger "train_log" defined at the beginning of this program
        StatsHandler(name="train_log", tag_name="train_loss", output_transform=from_engine(["loss"], first=True)),
        TensorBoardStatsHandler(log_dir="./runs/", tag_name="train_loss", output_transform=from_engine(["loss"], first=True)),
        CheckpointSaver(save_dir="./runs/", save_dict={"net": net, "opt": opt}, save_interval=1, epoch_level=True),
    ]

    trainer = SupervisedTrainer(
        device=device,
        max_epochs=500,
        train_data_loader=train_loader,
        network=net,
        optimizer=opt,
        loss_function=loss,
        inferer=SimpleInferer(),
        postprocessing=train_post_transforms,
        key_train_metric={"train_acc": Accuracy(output_transform=from_engine(["pred", "mask"]))},
        train_handlers=train_handlers,
        # if no FP16 support in GPU or PyTorch version < 1.6, will not enable AMP training
        amp=True if monai.utils.get_torch_version_tuple() >= (1, 6) else False,
    )
    # set initialized trainer for "early stop" handlers
    val_handlers[0].set_trainer(trainer=trainer)
    train_handlers[0].set_trainer(trainer=trainer)
    trainer.run()


if __name__ == "__main__":
    main(train_dir)

MONAI version: 0.9.dev2209
Numpy version: 1.21.5
Pytorch version: 1.10.2
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 9db6e543d6090a3256f20695c1d3224df8cbbc0e
MONAI __file__: C:\Users\Hripsime\anaconda3\envs\abys\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.8.0
gdown version: 4.2.1
TorchVision version: 0.11.3
tqdm version: 4.63.0
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.4.1
einops version: 0.4.0
transformers version: 4.16.2
mlflow version: 1.23.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



Loading dataset: 100%|█████████████████████████████████████████████████████████████████| 10/10 [01:29<00:00,  8.99s/it]


INFO:ignite.engine.engine.SupervisedTrainer:Engine run resuming from iteration 0, epoch 0 until 500 epochs
ERROR:ignite.engine.engine.SupervisedTrainer:Current run is terminating due to exception: 'NoneType' object has no attribute 'shape'
2022-04-12 20:21:42,026 - ERROR - Exception: 'NoneType' object has no attribute 'shape'
Traceback (most recent call last):
  File "C:\Users\Hripsime\anaconda3\envs\abys\lib\site-packages\ignite\engine\engine.py", line 840, in _run_once_on_dataset
    self.state.output = self._process_function(self, self.state.batch)
  File "C:\Users\Hripsime\anaconda3\envs\abys\lib\site-packages\monai\engines\trainer.py", line 204, in _iteration
    _compute_pred_loss()
  File "C:\Users\Hripsime\anaconda3\envs\abys\lib\site-packages\monai\engines\trainer.py", line 192, in _compute_pred_loss
    engine.state.output[Keys.LOSS] = self.loss_function(engine.state.output[Keys.PRED], targets).mean()
  File "C:\Users\Hripsime\anaconda3\envs\abys\lib\site-packages\torch\nn\mo

AttributeError: 'NoneType' object has no attribute 'shape'